## Ridge regression and classification
https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification

## Training and Logging

In [1]:
X = [[0, 0], [0, 0], [1, 1]]
X

[[0, 0], [0, 0], [1, 1]]

In [2]:
y = [0, .1, 1]
y

[0, 0.1, 1]

In [3]:
import getpass

import mlflow
import rikai
from sklearn.linear_model import Ridge


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)


# train a model
model = Ridge(alpha=0.5)
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_ridge"
    rikai.mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name=registered_model_name,
        model_type="rikai_sklearn.Ridge",
    )


/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/1a44acf611c08f97d7b24a8282221e4a93cb9cdb/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2022/11/14 10:57:15 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_ridge' already exists. Creating a new version of this model...
2022/11/14 10:57:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_ridge, version 4
Created version '4' of model 'da_ridge'.
/Users/da/.cache/pants/named_caches/pex_root/venvs/97296c1a2aa5705046eb53ad742c9a9d598e9ce1/1a44acf611c08f97d7b24a8282221e4a93cb9cdb/lib/python3.8/site-packages/rikai/spark/sql/codegen/mlflow_logger.py:183: UserWarning: va

## Apply the model on large scale dataset

In [4]:
from example import spark

:: loading settings :: url = jar:file:/Users/da/.cache/pants/named_caches/pex_root/installed_wheels/fcaa57f02b90be772d50778078fc41c3660d5a6c43218e45b2c2aef2ec8e9d58/pyspark-3.2.2-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/da/.ivy2/cache
The jars for the packages stored in: /Users/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-03928e25-f04e-492a-9b72-232cda4a5f3b;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.1.14 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 235ms :: artifacts dl 9ms
	:: modules in use:
	ai.eto#rikai_2.12;0.1.14 from central in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from spark-list in [default]
	org.x

In [5]:
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


+----------------+------+------------------+-------+
|name            |flavor|uri               |options|
+----------------+------+------------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_ridge|       |
+----------------+------+------------------+-------+



In [6]:
df = spark.range(10).selectExpr("id as x0", "id+1 as x1")
df.createOrReplaceTempView("tbl_X")

result = spark.sql(f"""
select ML_PREDICT(mlflow_sklearn_m, array(x0, x1)) from tbl_X
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: float (nullable = true)



,mlflow_sklearn_m
0,0.481818
1,1.172727
2,1.863636
3,2.554545
4,3.245455
5,3.936364
6,4.627273
7,5.318182
8,6.009091
9,6.700000
